# Predicción de supervivencia de pacientes con insuficiencia cardiaca


## Cargando Datos

In [ ]:
from google.colab import files

#Cargar el archivo 'heart_failure_clinical_records_dataset.csv' el cual se puede obtener en
#https://archive.ics.uci.edu/dataset/519/heart+failure+clinical+records
uploaded = files.upload()


Saving heart_failure_clinical_records_dataset.csv to heart_failure_clinical_records_dataset (1).csv


In [ ]:
#Importamos las paqueterías
import pandas as pd
import numpy as np
#Nos permitirán hacer unas graficas sobre la curva de aprendizaje de la red
import matplotlib.pyplot as plt
#Nos ayuda a hacer la validación cruzada
from sklearn.model_selection import RepeatedKFold,ShuffleSplit
#Nos ayuda a hacer mediciones sobre nuestros modelos
from sklearn.metrics import confusion_matrix,accuracy_score
#Función sigmoide logística
from scipy.special import expit
#Usaremos la paquetería scikitlearn que tiene integrado SVM lineal/no lineal y Regresión logística
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,precision_recall_curve,auc

#Cargamos nuestros datos en formato DataFrame
heart_df= pd.read_csv('heart_failure_clinical_records_dataset.csv')
#Visualizamos nuestros datos
heart_df.head(299)

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0


Realizamos un tratamiento a los datos extrayendo las variables booleanas y reemplazando los 0's por -1's

In [ ]:
#Redefinimos las variables booleanas
heart_df.loc[heart_df['anaemia']==0, 'anaemia']=-1 #fila 2 Anemia
heart_df.loc[heart_df['diabetes']==0, 'diabetes']=-1  #fila 4 Diabetes
heart_df.loc[heart_df['high_blood_pressure']==0, 'high_blood_pressure']=-1  #fila 6 Presion
heart_df.loc[heart_df['sex']==0, 'sex']=-1  #fila 8 Sexo
heart_df.loc[heart_df['smoking']==0, 'smoking']=-1  #fila 10 Fumador

#Extraemos las caracteristicas
X=np.transpose(heart_df.iloc[:,0:12].to_numpy())
X=X.astype(float)

#Almacenamos las variables que son binarias
Anemia=X[1,:].copy()
Diabetes=X[3,:].copy()
Presion=X[5,:].copy()
Sexo=X[9,:].copy()
Fumador=X[10,:].copy()

#Extraemos las etiquetas de clase
Y=heart_df['DEATH_EVENT'].to_numpy() #convertimos a vector
Y=Y.reshape(1,Y.shape[0]) #Redefine el tamaño del objeto
Y=Y.astype(float) #Convierte a valor punto flotante



Generamos las divisiones y repeticiones para nuestra validación cruzada

In [ ]:
#Generamos desde el inicio los split para la validación cruzada
random_state=13440 #Semilla aleatoria

# Se realizan las divisiones de cada una de las validaciones cruzadas
kf = RepeatedKFold(n_splits=5, n_repeats=20, random_state=random_state) #K-Fold CV
rcv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=random_state) #Random CV

En las siguientes listas almacenamreos las medidas presentadas en el articulo, por cada método

In [ ]:
MODELOS=['ANN SIGMOIDE', 'ANN TANH', 'ANN RELU',  'ANN 2 CAPAS', 'SVM LINEAL', 'SVM POLINOMIAL DEG 3', 'SVM GAUSSIANO', 'SVM SIGMOIDE', 'REGRESION LOG']

In [ ]:
ROC_AUC_KF=[]
PR_AUC_KF=[]
ACCURACY_KF=[]
TP_RATE_KF=[]
TN_RATE_KF=[]
F1_SCORE_KF=[]
MATTHEWS_C_KF=[]

ROC_AUC_CV=[]
PR_AUC_CV=[]
ACCURACY_CV=[]
TP_RATE_CV=[]
TN_RATE_CV=[]
F1_SCORE_CV=[]
MATTHEWS_C_CV=[]


Definimos la función de estandarización

In [ ]:
#Función de estandarización
#X:= (No. de datos, No. de caracteristicas)
def Estandarizar(X):
  X_mean=[]
  for i in range(X.shape[1]):
    X_mean.append(np.mean(X[:,i]))
  X_mean=np.array(X_mean)
  X_mean=X_mean.reshape(1,X_mean.shape[0])
  S=((X-X_mean).T@(X-X_mean))/X.shape[0]

  Var=np.array([S[i,i] for i in range(X.shape[1])])
  std=np.sqrt(Var)
  #Se extrae los datos estandarizados, la media y el vector con las desviaciones estandar por caracteristica
  #como un diccionario para su posterior uso
  return {
          'Xstd': (X-X_mean)/std,
          'media':X_mean,
          'desviacion': std.reshape(1,std.shape[0])

          }

Tenemos variables booleanas, así que hay que reemplazarlas

In [ ]:
#Se debe realizar una vez se han estandarizado los datos

def reemplazo(X,index):
  if type(index)!=type(np.zeros(10)):
    index=np.arange(X.shape[1])
  X[1,:]=Anemia[index]
  X[3,:]=Diabetes[index]
  X[5,:]=Presion[index]
  X[9,:]=Sexo[index]
  X[10,:]=Fumador[index]
  return X

## Entrenamiento del Modelo de una RNA con una capa oculta (Shallow Neural Network) con *tanh*, *ReLU* o *sigmoide logistica*.

En este modelo empleamos la función tanh para la capa oculta

In [ ]:
#Definimos en este apartado la derivada de la función ReLU

def devReLU(x):
  x[x<=0] = 0
  x[x>0] = 1
  return x

In [ ]:
# Las entradas deben de ser del tipo
#(Datos, Etiquetas, taza de aprendizaje, iteraciones, neuronas_Capa1, función de activación)
# En el caso de función acepta entradas 'ReLU', 'tanh', 'sigmoide'

#IMPORTANTE
#Datos:=(No. caracteristicas, No. de datos)
#Etiquetas debe ser un vector (no lista)

def shallow_NN( X, Y, lr, epochs, nh1, funcion):
  # Numero de observaciones
  m = X.shape[1]

  # Capas
  n0 = X.shape[0] #Caracteristicas
  n1 = nh1 #Neuronas en la capa oculta
  n2 = 1 #Neuronas en la capa de salida (en el caso binario es solo una)

  #Fijamos la semilla
  np.random.seed(1564)

  # Condiciones iniciales
  A0 = X
  W1 = np.random.randn(n1 ,n0)*0.1
  b1 = np.zeros((n1,1))

  W2 = np.random.randn(n2 ,n1)*0.01
  b2 = np.zeros((n2,1))*0.01

  loss = []

  #Determinamos la función de activación y por tanto la estructura de la red
  if funcion == 'tanh':
    for i in range(epochs):
      # Forward propagation
      Z1 = W1 @ A0 + b1
      A1 = np.tanh(Z1)

      Z2 = W2 @ A1 + b2
      A2 = expit(Z2)

      # Backward propagation
      dZ2 = A2 - Y
      dW2 = (1.0/m) * ( dZ2 @ A1.T )
      db2 = (1.0/m) * np.sum( dZ2, axis=1, keepdims=True )

      dZ1 =  (W2.T @ dZ2) * ( 1.0 - A1**2)
      dW1 = (1.0/m) * ( dZ1 @ A0.T )
      db1 = (1.0/m) * np.sum( dZ1, axis=1, keepdims=True )

      # Actualización de los pesos
      W1 = W1 - lr*dW1
      b1 = b1 - lr*db1

      W2 = W2 - lr*dW2
      b2 = b2 - lr*db2

      loss.append( -(1.0/m)*np.sum( Y*np.log(A2) + (1.0-Y)*np.log( 1.0 - A2)) )

  elif funcion == 'sigmoide':

    for i in range(epochs):

      # Forward propagation
      Z1 = W1 @ A0 + b1
      A1 = expit(Z1)

      Z2 = W2 @ A1 + b2
      A2 = expit(Z2)

      # Backward propagation
      dZ2 = A2 - Y
      dW2 = (1.0/m) * ( dZ2 @ A1.T )
      db2 = (1.0/m) * np.sum( dZ2, axis=1, keepdims=True )


      dZ1 =  (W2.T @ dZ2) *A1*( 1.0 - A1)
      dW1 = (1.0/m) * ( dZ1 @ A0.T )
      db1 = (1.0/m) * np.sum( dZ1, axis=1, keepdims=True )


      # Actualización de los pesos
      W1 = W1 - lr*dW1
      b1 = b1 - lr*db1

      W2 = W2 - lr*dW2
      b2 = b2 - lr*db2

      loss.append( -(1.0/m)*np.sum( Y*np.log(A2) + (1.0-Y)*np.log( 1.0 - A2)) )

  elif funcion == 'ReLU':

    for i in range(epochs):

      # Forward propagation
      Z1 = W1 @ A0 + b1
      A1 = np.maximum(0,Z1)

      Z2 = W2 @ A1 + b2
      A2 = expit(Z2)

      # Backward propagation
      dZ2 = A2 - Y
      dW2 = (1.0/m) * ( dZ2 @ A1.T )
      db2 = (1.0/m) * np.sum( dZ2, axis=1, keepdims=True )


      dZ1 =  (W2.T @ dZ2) *devReLU(A1)
      dW1 = (1.0/m) * ( dZ1 @ A0.T )
      db1 = (1.0/m) * np.sum( dZ1, axis=1, keepdims=True )


      # Actualización de los pesos
      W1 = W1 - lr*dW1
      b1 = b1 - lr*db1

      W2 = W2 - lr*dW2
      b2 = b2 - lr*db2

      loss.append( -(1.0/m)*np.sum( Y*np.log(A2) + (1.0-Y)*np.log( 1.0 - A2)) )

  #print(f'La función de costo en la iteración {epochs} es {loss[-1]}')
  return {
        'W1': W1,
        'b1': b1,
        'W2': W2,
        'b2': b2,
        'loss':loss,
        'funcion':funcion
        }

Con los parametros obtenidos generamos nuestro clasificador.

In [ ]:
def predict( results, X, prob=False):

  funcion=results['funcion']
  A0 = X
  W1 = results['W1']
  b1 = results['b1']

  W2 = results['W2']
  b2 = results['b2']


  if funcion == 'tanh':
    # Forward propagation
    Z1 = W1 @ A0 + b1
    A1 = np.tanh(Z1)

  elif funcion == 'ReLU':
    # Forward propagation
    Z1 = W1 @ A0 + b1
    A1 = np.maximum(0,Z1)

  elif funcion == 'sigmoide':
    # Forward propagation
    Z1 = W1 @ A0 + b1
    A1 = expit(Z1)

  Z2 = W2 @ A1 + b2
  A2 = expit(Z2)

  if prob==False:
    yhat = np.around(A2)
  else:
    yhat = A2

  return yhat

### K-Fold Cross Validation

Aquí comenzamos con la validación cruzada K-Fold. Hemos empleado 20 validaciones 5-Fold para este dataset. Aplicamos el proceso solo a datos estandarizados.

In [ ]:
for funcion in ['sigmoide','tanh', 'ReLU']:

  if funcion=='tanh':
    epochsRN=1950
    nh1=50
    learn_ratio=0.005
  elif funcion=='ReLU':
    epochsRN=1050
    nh1=140
    learn_ratio=0.0125
  elif funcion=='sigmoide':
    epochsRN=500
    nh1=40
    learn_ratio=0.1
  # Listas para almacenar los valores verdaderos y predichos
  y_true = []
  y_pred = []

  # Listas para guardar las mediciones
  F1_SCORE=[]
  MATTHEWS_C=[]
  TP_RATE=[]
  TN_RATE=[]
  ROC_AUC=[]
  PR_AUC=[]
  ACCURACY=[]

  print(f"Epochs: {epochsRN}, Neuronas: {nh1}, Lear ratio: {learn_ratio}")

  # Realizar la validación cruzada K-Fold
  for train_index, test_index in kf.split(X[0,:]):
    X_train, X_test = X[:,train_index], X[:,test_index]
    y_train, y_test = Y[:,train_index], Y[:,test_index]

    # Se estandarizan los datos respecto al conjunto de datos de entrenamiento.
    Datos_estandarizados=Estandarizar(X_train.T)
    Datos_Test_estandarizados=(X_test-Datos_estandarizados['media'].T)/Datos_estandarizados['desviacion'].T


    # Datos reemplazados en las variables binarias
    Datos=reemplazo(Datos_estandarizados['Xstd'].T,train_index)
    Datos_Test=reemplazo(Datos_Test_estandarizados,test_index)
    #Entrenamiento de la red neuronal
    results = shallow_NN(Datos, y_train, learn_ratio, epochsRN, nh1,funcion)

    # Predecir las muestras de prueba
    y_pred_fold = predict(results,Datos_Test)

    #Calculamos localmente la matriz de confusión con el fin de hacer las mediciones
    CML= confusion_matrix(y_test[0,:], y_pred_fold[0,:])
    TP=CML[0,0]
    FP=CML[1,0]
    TN=CML[1,1]
    FN=CML[0,1]

    #Estimamos las probabilidades de cada dato
    y_prob=predict(results, Datos_Test, prob=True)

    #Determinamos la curva precision-recall para calcular el PR AUC
    precision, recall,_=precision_recall_curve(y_test[0,:],y_prob[0,:])

    #Calculamos todas nuestras mediciones
    F1_SCORE.append(2*TP/(2*TP+FP+FN))
    MATTHEWS_C.append((TP*TN-FP*FN)/np.sqrt((TP+FP)*(FN+TN)*(FP+TN)*(FN+TP)))
    ACCURACY.append((TP+TN)/(TP+FN+FP+TN))
    TP_RATE.append(TP/(TP+FN))
    TN_RATE.append(TN/(TN+FP))
    ROC_AUC.append(roc_auc_score(y_test[0,:], y_prob[0,:]))
    PR_AUC.append(auc(recall,precision))

    y_pred.extend(np.transpose(y_pred_fold))
    y_true.extend(np.transpose(y_test))

  # Convertir las listas a arrays de NumPy

  y_pred =np.transpose(np.array(y_pred))
  y_true =np.transpose(np.array(y_true))

  # Calcular la matriz de confusión
  conf_matrix = confusion_matrix(y_true[0,:],y_pred[0,:])

  # Convertir la matriz de confusión a un DataFrame para mejor formato
  conf_matrix_df = pd.DataFrame(
  conf_matrix,
  index=['Supervivencia', 'Muerte'],
  columns=['Pred. Supervivencia', 'Pred. Muerte']
  )

  # Agregar totales por fila y por columna
  conf_matrix_df.loc['Total fila'] = conf_matrix_df.sum(axis=0)
  conf_matrix_df['Total columna'] = conf_matrix_df.sum(axis=1)

  # Imprimir la matriz de confusión
  print(f"Matriz de confusión K-Fold CV para una Red Neuronal con 1 capa oculta y función {funcion}:" )
  print(conf_matrix_df)

  # Calcular la proporción de clasificación errónea
  accuracy = accuracy_score(np.transpose(y_true), np.transpose(y_pred))
  misclass_rate = 1 - accuracy
  print("Prop. error clas. global: %.3f" %  misclass_rate)
  print("Media de 100 simulaciones de las mediciones")
  print(f"F1 score: {np.mean(F1_SCORE)}")
  print(f"Correlación de Matthews: {np.mean(MATTHEWS_C)}")
  print(f"Razón de verdaderos positivos (TP-rate): {np.mean(TP_RATE)}")
  print(f"Razón de verdaderos negativos (TN-rate): {np.mean(TN_RATE)}")
  print(f"Precisión: {np.mean(ACCURACY)}")
  print(f"Área bajo la curva ROC (ROC AUC): {np.mean(ROC_AUC)}")
  print(f"Área bajo la curva Precision-Recall (PR AUC): {np.mean(PR_AUC)}")
  print("")

  ROC_AUC_KF.append(np.mean(ROC_AUC))
  TP_RATE_KF.append(np.mean(TP_RATE))
  TN_RATE_KF.append(np.mean(TN_RATE))
  PR_AUC_KF.append(np.mean(PR_AUC))
  F1_SCORE_KF.append(np.mean(F1_SCORE))
  ACCURACY_KF.append(np.mean(ACCURACY))
  MATTHEWS_C_KF.append(np.mean(MATTHEWS_C))

Epochs: 500, Neuronas: 40, Lear ratio: 0.1
Matriz de confusión K-Fold CV para una Red Neuronal con 1 capa oculta y función sigmoide:
               Pred. Supervivencia  Pred. Muerte  Total columna
Supervivencia                 3666           394           4060
Muerte                         659          1261           1920
Total fila                    4325          1655           5980
Prop. error clas. global: 0.176
Media de 100 simulaciones de las mediciones
F1 score: 0.8734111863673952
Correlación de Matthews: 0.584672038832651
Razón de verdaderos positivos (TP-rate): 0.9032059480756803
Razón de verdaderos negativos (TN-rate): 0.6573698602959236
Precisión: 0.8239322033898304
Área bajo la curva ROC (ROC AUC): 0.8708734062786364
Área bajo la curva Precision-Recall (PR AUC): 0.766369702123855

Epochs: 1950, Neuronas: 50, Lear ratio: 0.005
Matriz de confusión K-Fold CV para una Red Neuronal con 1 capa oculta y función tanh:
               Pred. Supervivencia  Pred. Muerte  Total columna

### Random Cross Validation

In [ ]:
for funcion in ['sigmoide', 'tanh', 'ReLU' ]:

  if funcion=='tanh':
    epochsRN=1950
    nh1=50
    learn_ratio=0.005
  elif funcion=='ReLU':
    epochsRN=1050
    nh1=140
    learn_ratio=0.0125
  elif funcion=='sigmoide':
    epochsRN=500
    nh1=40
    learn_ratio=0.1

  # Listas para almacenar los valores verdaderos y predichos
  y_true = []
  y_pred = []

  # Listas para guardar las mediciones
  F1_SCORE=[]
  MATTHEWS_C=[]
  TP_RATE=[]
  TN_RATE=[]
  ROC_AUC=[]
  PR_AUC=[]
  ACCURACY=[]

  # Realizar la validación cruzada K-Fold
  for train_index, test_index in rcv.split(X[0,:]):
    X_train, X_test = X[:,train_index], X[:,test_index]
    y_train, y_test = Y[:,train_index], Y[:,test_index]

    # Se estandarizan los datos respecto al conjunto de datos de entrenamiento.
    Datos_estandarizados=Estandarizar(X_train.T)
    Datos_Test_estandarizados=(X_test-Datos_estandarizados['media'].T)/Datos_estandarizados['desviacion'].T

    # Datos reemplazados en las variables binarias
    Datos=reemplazo(Datos_estandarizados['Xstd'].T,train_index)
    Datos_Test=reemplazo(Datos_Test_estandarizados,test_index)
    #Entrenamiento de la red neuronal
    results = shallow_NN(Datos, y_train, learn_ratio, epochsRN, nh1,funcion)

    # Predecir las muestras de prueba
    y_pred_fold = predict(results,Datos_Test)

    CML= confusion_matrix(y_test[0,:], y_pred_fold[0,:])
    TP=CML[0,0]
    FP=CML[1,0]
    TN=CML[1,1]
    FN=CML[0,1]

    #Estimamos las probabilidades de cada dato
    y_prob=predict(results, Datos_Test, prob=True)

    #Determinamos la curva precision-recall para calcular el PR AUC
    precision, recall,_=precision_recall_curve(y_test[0,:],y_prob[0,:])

    #Calculamos todas nuestras mediciones
    F1_SCORE.append(2*TP/(2*TP+FP+FN))
    MATTHEWS_C.append((TP*TN-FP*FN)/np.sqrt((TP+FP)*(FN+TN)*(FP+TN)*(FN+TP)))
    ACCURACY.append((TP+TN)/(TP+FN+FP+TN))
    TP_RATE.append(TP/(TP+FN))
    TN_RATE.append(TN/(TN+FP))
    ROC_AUC.append(roc_auc_score(y_test[0,:], y_prob[0,:]))
    PR_AUC.append(auc(recall,precision))

    y_pred.extend(np.transpose(y_pred_fold))
    y_true.extend(np.transpose(y_test))
  # Convertir las listas a arrays de NumPy

  y_pred =np.transpose(np.array(y_pred))
  y_true =np.transpose(np.array(y_true))

  # Calcular la matriz de confusión
  conf_matrix = confusion_matrix(y_true[0,:],y_pred[0,:])

  # Convertir la matriz de confusión a un DataFrame para mejor formato
  conf_matrix_df = pd.DataFrame(
  conf_matrix,
  index=['Supervivencia', 'Muerte'],
  columns=['Pred. Supervivencia', 'Pred. Muerte']
  )

  # Agregar totales por fila y por columna
  conf_matrix_df.loc['Total fila'] = conf_matrix_df.sum(axis=0)
  conf_matrix_df['Total columna'] = conf_matrix_df.sum(axis=1)

  # Imprimir la matriz de confusión
  print(f"Matriz de confusión K-Fold CV para una Red Neuronal con 1 capa oculta y función {funcion}:" )
  print(conf_matrix_df)

  # Calcular la proporción de clasificación errónea
  accuracy = accuracy_score(np.transpose(y_true), np.transpose(y_pred))
  misclass_rate = 1 - accuracy
  print("Prop. error clas. global: %.3f" %  misclass_rate)
  print("Media de 100 simulaciones de las mediciones")
  print(f"F1 score: {np.mean(F1_SCORE)}")
  print(f"Correlación de Matthews: {np.mean(MATTHEWS_C)}")
  print(f"Razón de verdaderos positivos (TP-rate): {np.mean(TP_RATE)}")
  print(f"Razón de verdaderos negativos (TN-rate): {np.mean(TN_RATE)}")
  print(f"Precisión: {np.mean(ACCURACY)}")
  print(f"Área bajo la curva ROC (ROC AUC): {np.mean(ROC_AUC)}")
  print(f"Área bajo la curva Precision-Recall (PR AUC): {np.mean(PR_AUC)}")
  print("")

  ROC_AUC_CV.append(np.mean(ROC_AUC))
  TP_RATE_CV.append(np.mean(TP_RATE))
  TN_RATE_CV.append(np.mean(TN_RATE))
  PR_AUC_CV.append(np.mean(PR_AUC))
  F1_SCORE_CV.append(np.mean(F1_SCORE))
  ACCURACY_CV.append(np.mean(ACCURACY))
  MATTHEWS_C_CV.append(np.mean(MATTHEWS_C))


Matriz de confusión K-Fold CV para una Red Neuronal con 1 capa oculta y función sigmoide:
               Pred. Supervivencia  Pred. Muerte  Total columna
Supervivencia                 3662           434           4096
Muerte                         656          1248           1904
Total fila                    4318          1682           6000
Prop. error clas. global: 0.182
Media de 100 simulaciones de las mediciones
F1 score: 0.8697547695868292
Correlación de Matthews: 0.5687512981931374
Razón de verdaderos positivos (TP-rate): 0.8947497551138429
Razón de verdaderos negativos (TN-rate): 0.6548677240051045
Precisión: 0.8183333333333331
Área bajo la curva ROC (ROC AUC): 0.8648564497230362
Área bajo la curva Precision-Recall (PR AUC): 0.7547082452285193

Matriz de confusión K-Fold CV para una Red Neuronal con 1 capa oculta y función tanh:
               Pred. Supervivencia  Pred. Muerte  Total columna
Supervivencia                 3634           462           4096
Muerte                

## Entrenamiento del modelo de una Red Neuronal con 2 capas ocultas (Deep Neural Network)

In [ ]:
def Deep_NN( X, Y, lr, epochs, nh1, nh2):
  # Numero de observaciones
  m = X.shape[1]

  # Layers: input, two hidden
  n0 = X.shape[0]
  n1 = nh1
  n2 = nh2
  n3 = 1

  #Fijamos la semilla
  np.random.seed(13440)

  # Condiciones iniciales
  A0 = X
  W1 = np.random.randn(n1 ,n0)
  b1 = np.zeros((n1,1))

  W2 = np.random.randn(n2 ,n1)
  b2 = np.zeros((n2,1))

  W3 = np.random.randn(n3 ,n2)
  b3 = np.zeros((n3,1))

  loss = []
  for i in range(epochs):

    # Forward propagation
    Z1 = W1 @ A0 + b1
    A1 = expit(Z1)

    Z2 = W2 @ A1 + b2
    A2 = expit(Z2)

    Z3 = W3 @ A2 + b3
    A3 = expit(Z3)

    # Backward propagation
    dZ3 = A3 - Y
    dW3 = (1.0/m) * ( dZ3 @ A2.T )
    db3 = (1.0/m) * np.sum( dZ3, axis=1, keepdims=True )

    dZ2 =  (W3.T @ dZ3)*A2*(1-A2)
    dW2 = (1.0/m) * ( dZ2 @ A1.T )
    db2 = (1.0/m) * np.sum( dZ2, axis=1, keepdims=True )

    dZ1 =  (W2.T @ dZ2) *A1*(1-A1)
    dW1 = (1.0/m) * ( dZ1 @ A0.T )
    db1 = (1.0/m) * np.sum( dZ1, axis=1, keepdims=True )

    # Actualización de los pesos
    W1 = W1 - lr*dW1
    b1 = b1 - lr*db1

    W2 = W2 - lr*dW2
    b2 = b2 - lr*db2

    W3 = W3 - lr*dW3
    b3 = b3 - lr*db3

    loss.append( -(1.0/m)*np.sum( Y*np.log(A3) + (1.0-Y)*np.log( 1.0 - A3)) )

  #print(f'La función de costo en la iteración {epochs} es {loss[-1]}')
  return {
        'W1': W1,
        'b1': b1,
        'W2': W2,
        'b2': b2,
        'W3': W3,
        'b3': b3,
        'loss':loss
        }

In [ ]:
#Definimos los parametros
nh1=40
nh2=50
epochsRN=3000
learn_ratio=0.1

In [ ]:
def predict( results, X, prob=False):

  A0 = X
  W1 = results['W1']
  b1 = results['b1']

  W2 = results['W2']
  b2 = results['b2']

  W3 = results['W3']
  b3 = results['b3']

  # Forward propagation
  Z1 = W1 @ A0 + b1
  A1 = expit(Z1)

  Z2 = W2 @ A1 + b2
  A2 = expit(Z2)

  Z3 = W3 @ A2 + b3
  A3 = expit(Z3)

  if prob==False:
    yhat = np.around(A3)
  else:
    yhat=A3

  return yhat

#### K-Fold Cross Validation

In [ ]:
# Listas para almacenar los valores verdaderos y predichos
y_true = []
y_pred = []

# Listas para guardar las mediciones
F1_SCORE=[]
MATTHEWS_C=[]
TP_RATE=[]
TN_RATE=[]
ROC_AUC=[]
PR_AUC=[]
ACCURACY=[]

# Realizar la validación cruzada K-Fold
for train_index, test_index in kf.split(X[0,:]):
  X_train, X_test = X[:,train_index], X[:,test_index]
  y_train, y_test = Y[:,train_index], Y[:,test_index]

  # Se estandarizan los datos respecto al conjunto de datos de entrenamiento.
  Datos_estandarizados=Estandarizar(X_train.T)
  Datos_Test_estandarizados=(X_test-Datos_estandarizados['media'].T)/Datos_estandarizados['desviacion'].T

  # Datos reemplazados en las variables binarias
  Datos=reemplazo(Datos_estandarizados['Xstd'].T,train_index)
  Datos_Test=reemplazo(Datos_Test_estandarizados,test_index)

  # Entrenar la red neuronal
  results = Deep_NN(Datos, y_train, learn_ratio, epochsRN, nh1,nh2)

  # Predecir las muestras de prueba
  y_pred_fold = predict(results,Datos_Test)

  CML= confusion_matrix(y_test[0,:], y_pred_fold[0,:])
  TP=CML[0,0]
  FP=CML[1,0]
  TN=CML[1,1]
  FN=CML[0,1]

  #Estimamos las probabilidades de cada dato
  y_prob=predict(results, Datos_Test, prob=True)

  #Determinamos la curva precision-recall para calcular el PR AUC
  precision, recall,_=precision_recall_curve(y_test[0,:],y_prob[0,:])

  #Calculamos todas nuestras mediciones
  F1_SCORE.append(2*TP/(2*TP+FP+FN))
  MATTHEWS_C.append((TP*TN-FP*FN)/np.sqrt((TP+FP)*(FN+TN)*(FP+TN)*(FN+TP)))
  ACCURACY.append((TP+TN)/(TP+FN+FP+TN))
  TP_RATE.append(TP/(TP+FN))
  TN_RATE.append(TN/(TN+FP))
  ROC_AUC.append(roc_auc_score(y_test[0,:], y_prob[0,:]))
  PR_AUC.append(auc(recall,precision))

  y_pred.extend(np.transpose(y_pred_fold))
  y_true.extend(np.transpose(y_test))

# Convertir las listas a arrays de NumPy

y_pred =np.transpose(np.array(y_pred))
y_true =np.transpose(np.array(y_true))

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_pred[0,:], y_true[0,:])

# Convertir la matriz de confusión a un DataFrame para mejor formato
conf_matrix_df = pd.DataFrame(
conf_matrix,
index=['Supervivencia', 'Muerte'],
columns=['Pred. Supervivencia', 'Pred. Muerte']
)

# Agregar totales por fila y por columna
conf_matrix_df.loc['Total fila'] = conf_matrix_df.sum(axis=0)
conf_matrix_df['Total columna'] = conf_matrix_df.sum(axis=1)

# Imprimir la matriz de confusión
print("Matriz de confusión K-Fold CV para una Red Neuronal con 2 capas oculta:")
print(conf_matrix_df)

# Calcular la proporción de clasificación errónea
accuracy = accuracy_score(np.transpose(y_true), np.transpose(y_pred))
misclass_rate = 1 - accuracy
print("Prop. error clas. global: %.3f" %  misclass_rate)
print("Media de 100 simulaciones de las mediciones")
print(f"F1 score: {np.mean(F1_SCORE)}")
print(f"Correlación de Matthews: {np.mean(MATTHEWS_C)}")
print(f"Razón de verdaderos positivos (TP-rate): {np.mean(TP_RATE)}")
print(f"Razón de verdaderos negativos (TN-rate): {np.mean(TN_RATE)}")
print(f"Precisión: {np.mean(ACCURACY)}")
print(f"Área bajo la curva ROC (ROC AUC): {np.mean(ROC_AUC)}")
print(f"Área bajo la curva Precision-Recall (PR AUC): {np.mean(PR_AUC)}")
print("")

ROC_AUC_KF.append(np.mean(ROC_AUC))
TP_RATE_KF.append(np.mean(TP_RATE))
TN_RATE_KF.append(np.mean(TN_RATE))
PR_AUC_KF.append(np.mean(PR_AUC))
F1_SCORE_KF.append(np.mean(F1_SCORE))
ACCURACY_KF.append(np.mean(ACCURACY))
MATTHEWS_C_KF.append(np.mean(MATTHEWS_C))

Matriz de confusión K-Fold CV para una Red Neuronal con 2 capas oculta:
               Pred. Supervivencia  Pred. Muerte  Total columna
Supervivencia                 3483           805           4288
Muerte                         577          1115           1692
Total fila                    4060          1920           5980
Prop. error clas. global: 0.231
Media de 100 simulaciones de las mediciones
F1 score: 0.8330368930653061
Correlación de Matthews: 0.45681136280352264
Razón de verdaderos positivos (TP-rate): 0.8582774247666548
Razón de verdaderos negativos (TN-rate): 0.5814838146149628
Precisión: 0.7688898305084746
Área bajo la curva ROC (ROC AUC): 0.836887373779058
Área bajo la curva Precision-Recall (PR AUC): 0.6967802540649302



#### Random Cross Validation

In [ ]:
# Listas para almacenar los valores verdaderos y predichos
y_true = []
y_pred = []

# Listas para guardar las mediciones
F1_SCORE=[]
MATTHEWS_C=[]
TP_RATE=[]
TN_RATE=[]
ROC_AUC=[]
PR_AUC=[]
ACCURACY=[]

# Realizar la validación cruzada K-Fold
for train_index, test_index in rcv.split(X[0,:]):
  X_train, X_test = X[:,train_index], X[:,test_index]
  y_train, y_test = Y[:,train_index], Y[:,test_index]

  # Se estandarizan los datos respecto al conjunto de datos de entrenamiento.
  Datos_estandarizados=Estandarizar(X_train.T)
  Datos_Test_estandarizados=(X_test-Datos_estandarizados['media'].T)/Datos_estandarizados['desviacion'].T

  # Datos reemplazados en las variables binarias
  Datos=reemplazo(Datos_estandarizados['Xstd'].T,train_index)
  Datos_Test=reemplazo(Datos_Test_estandarizados,test_index)

  # Entrenar la red neuronal
  results = Deep_NN(Datos, y_train, learn_ratio, epochsRN, nh1,nh2)

  # Predecir las muestras de prueba
  y_pred_fold = predict(results,Datos_Test)

  CML= confusion_matrix(y_test[0,:], y_pred_fold[0,:])
  TP=CML[0,0]
  FP=CML[1,0]
  TN=CML[1,1]
  FN=CML[0,1]

  #Estimamos las probabilidades de cada dato
  y_prob=predict(results, Datos_Test, prob=True)

  #Determinamos la curva precision-recall para calcular el PR AUC
  precision, recall,_=precision_recall_curve(y_test[0,:],y_prob[0,:])

  #Calculamos todas nuestras mediciones
  F1_SCORE.append(2*TP/(2*TP+FP+FN))
  MATTHEWS_C.append((TP*TN-FP*FN)/np.sqrt((TP+FP)*(FN+TN)*(FP+TN)*(FN+TP)))
  ACCURACY.append((TP+TN)/(TP+FN+FP+TN))
  TP_RATE.append(TP/(TP+FN))
  TN_RATE.append(TN/(TN+FP))
  ROC_AUC.append(roc_auc_score(y_test[0,:], y_prob[0,:]))
  PR_AUC.append(auc(recall,precision))

  y_pred.extend(np.transpose(y_pred_fold))
  y_true.extend(np.transpose(y_test))

# Convertir las listas a arrays de NumPy

y_pred =np.transpose(np.array(y_pred))
y_true =np.transpose(np.array(y_true))

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_pred[0,:], y_true[0,:])

# Convertir la matriz de confusión a un DataFrame para mejor formato
conf_matrix_df = pd.DataFrame(
conf_matrix,
index=['Supervivencia', 'Muerte'],
columns=['Pred. Supervivencia', 'Pred. Muerte']
)

# Agregar totales por fila y por columna
conf_matrix_df.loc['Total fila'] = conf_matrix_df.sum(axis=0)
conf_matrix_df['Total columna'] = conf_matrix_df.sum(axis=1)

# Imprimir la matriz de confusión
print("Matriz de confusión K-Fold CV para una Red Neuronal con 2 capas oculta:")
print(conf_matrix_df)

# Calcular la proporción de clasificación errónea
accuracy = accuracy_score(np.transpose(y_true), np.transpose(y_pred))
misclass_rate = 1 - accuracy
print("Prop. error clas. global: %.3f" %  misclass_rate)
print("Media de 100 simulaciones de las mediciones")
print(f"F1 score: {np.mean(F1_SCORE)}")
print(f"Correlación de Matthews: {np.mean(MATTHEWS_C)}")
print(f"Razón de verdaderos positivos (TP-rate): {np.mean(TP_RATE)}")
print(f"Razón de verdaderos negativos (TN-rate): {np.mean(TN_RATE)}")
print(f"Precisión: {np.mean(ACCURACY)}")
print(f"Área bajo la curva ROC (ROC AUC): {np.mean(ROC_AUC)}")
print(f"Área bajo la curva Precision-Recall (PR AUC): {np.mean(PR_AUC)}")
print("")

ROC_AUC_CV.append(np.mean(ROC_AUC))
TP_RATE_CV.append(np.mean(TP_RATE))
TN_RATE_CV.append(np.mean(TN_RATE))
PR_AUC_CV.append(np.mean(PR_AUC))
F1_SCORE_CV.append(np.mean(F1_SCORE))
ACCURACY_CV.append(np.mean(ACCURACY))
MATTHEWS_C_CV.append(np.mean(MATTHEWS_C))

Matriz de confusión K-Fold CV para una Red Neuronal con 2 capas oculta:
               Pred. Supervivencia  Pred. Muerte  Total columna
Supervivencia                 3501           810           4311
Muerte                         595          1094           1689
Total fila                    4096          1904           6000
Prop. error clas. global: 0.234
Media de 100 simulaciones de las mediciones
F1 score: 0.8315745414653621
Correlación de Matthews: 0.44383204409387
Razón de verdaderos positivos (TP-rate): 0.8546714026816334
Razón de verdaderos negativos (TN-rate): 0.574131565671507
Precisión: 0.7658333333333333
Área bajo la curva ROC (ROC AUC): 0.8342991787851194
Área bajo la curva Precision-Recall (PR AUC): 0.6899930379673925



## SVM lineal

In [ ]:
#Ajustamos el modelo SVM lineal
model=svm.SVC(kernel='linear')
#El SVM recibe etiquetas como una lista y datos en la forma (No. Datos, No. Caracteristicas)
Etiquetas=Y.T
Etiquetas=Etiquetas.ravel()
Datos=X.T

#### K-Fold Cross Validation

In [ ]:
y_true = []
y_pred = []

# Listas para guardar las mediciones
F1_SCORE=[]
MATTHEWS_C=[]
TP_RATE=[]
TN_RATE=[]
ROC_AUC=[]
PR_AUC=[]
ACCURACY=[]


# Realizar la validación cruzada K-Fold
for train_index, test_index in kf.split(X[0,:]):
  X_train, X_test = Datos[train_index,:], Datos[test_index,:]
  y_train, y_test = Etiquetas[train_index], Etiquetas[test_index]

  # Se estandarizan los datos respecto al conjunto de datos de entrenamiento.
  Datos_estandarizados=Estandarizar(X_train)
  Datos_Test_estandarizados=(X_test-Datos_estandarizados['media'])/Datos_estandarizados['desviacion']

  # Datos reemplazados en las variables binarias
  Datos_Train=reemplazo(Datos_estandarizados['Xstd'].T,train_index)
  Datos_Test=reemplazo(Datos_Test_estandarizados.T,test_index)

  #Entrenamiento del SVM lineal
  results = model.fit(Datos_Train.T, y_train)

  # Predecir las muestras de prueba
  y_pred_fold = results.predict(Datos_Test.T)

  CML= confusion_matrix(y_test, y_pred_fold)
  TP=CML[0,0]
  FP=CML[1,0]
  TN=CML[1,1]
  FN=CML[0,1]

  precision, recall,_=precision_recall_curve(y_test,results.decision_function(Datos_Test.T))

  #Calculamos todas nuestras mediciones
  F1_SCORE.append(2*TP/(2*TP+FP+FN))
  MATTHEWS_C.append((TP*TN-FP*FN)/np.sqrt((TP+FP)*(FN+TN)*(FP+TN)*(FN+TP)))
  ACCURACY.append((TP+TN)/(TP+FN+FP+TN))
  TP_RATE.append(TP/(TP+FN))
  TN_RATE.append(TN/(TN+FP))
  ROC_AUC.append(roc_auc_score(y_test, results.decision_function(Datos_Test.T)))
  PR_AUC.append(auc(recall,precision))

  y_pred.extend(y_pred_fold)
  y_true.extend(y_test)


# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_true,y_pred)

# Convertir la matriz de confusión a un DataFrame para mejor formato
conf_matrix_df = pd.DataFrame(
conf_matrix,
index=['Supervivencia', 'Muerte'],
columns=['Pred. Supervivencia', 'Pred. Muerte']
)

# Agregar totales por fila y por columna
conf_matrix_df.loc['Total fila'] = conf_matrix_df.sum(axis=0)
conf_matrix_df['Total columna'] = conf_matrix_df.sum(axis=1)

# Imprimir la matriz de confusión
print("Matriz de confusión K-Fold CV para SVM:" )
print(conf_matrix_df)

# Calcular la proporción de clasificación errónea
accuracy = accuracy_score(y_true,y_pred)
misclass_rate = 1 - accuracy
print("Prop. error clas. global: %.3f" %  misclass_rate)
print("Media de 100 simulaciones de las mediciones")
print(f"F1 score: {np.mean(F1_SCORE)}")
print(f"Correlación de Matthews: {np.mean(MATTHEWS_C)}")
print(f"Razón de verdaderos positivos (TP-rate): {np.mean(TP_RATE)}")
print(f"Razón de verdaderos negativos (TN-rate): {np.mean(TN_RATE)}")
print(f"Precisión: {np.mean(ACCURACY)}")
print(f"Área bajo la curva ROC (ROC AUC): {np.mean(ROC_AUC)} ")
print(f"Área bajo la curva precision-recall (PR AUC): {np.mean(PR_AUC)}")

ROC_AUC_KF.append(np.mean(ROC_AUC))
TP_RATE_KF.append(np.mean(TP_RATE))
TN_RATE_KF.append(np.mean(TN_RATE))
PR_AUC_KF.append(np.mean(PR_AUC))
F1_SCORE_KF.append(np.mean(F1_SCORE))
ACCURACY_KF.append(np.mean(ACCURACY))
MATTHEWS_C_KF.append(np.mean(MATTHEWS_C))

Matriz de confusión K-Fold CV para SVM:
               Pred. Supervivencia  Pred. Muerte  Total columna
Supervivencia                 3606           454           4060
Muerte                         637          1283           1920
Total fila                    4243          1737           5980
Prop. error clas. global: 0.182
Media de 100 simulaciones de las mediciones
F1 score: 0.8675810617855805
Correlación de Matthews: 0.5723131731256073
Razón de verdaderos positivos (TP-rate): 0.8885019769623577
Razón de verdaderos negativos (TN-rate): 0.666748262440361
Precisión: 0.8175790960451976
Área bajo la curva ROC (ROC AUC): 0.8646204382768004 
Área bajo la curva precision-recall (PR AUC): 0.7596463768585742


#### Random Cross Validation

In [ ]:
y_true = []
y_pred = []

# Listas para guardar las mediciones
F1_SCORE=[]
MATTHEWS_C=[]
TP_RATE=[]
TN_RATE=[]
ROC_AUC=[]
PR_AUC=[]
ACCURACY=[]

# Realizar la validación cruzada aleatoria
for train_index, test_index in rcv.split(X[0,:]):
  X_train, X_test = Datos[train_index,:], Datos[test_index,:]
  y_train, y_test = Etiquetas[train_index], Etiquetas[test_index]

  # Se estandarizan los datos respecto al conjunto de datos de entrenamiento.
  Datos_estandarizados=Estandarizar(X_train)
  Datos_Test_estandarizados=(X_test-Datos_estandarizados['media'])/Datos_estandarizados['desviacion']

  # Datos reemplazados en las variables binarias
  Datos_Train=reemplazo(Datos_estandarizados['Xstd'].T,train_index)
  Datos_Test=reemplazo(Datos_Test_estandarizados.T,test_index)
  #Entrenamiento del SVM lineal
  results = model.fit(Datos_Train.T, y_train)

  # Predecir las muestras de prueba
  y_pred_fold = results.predict(Datos_Test.T)

  CML= confusion_matrix(y_test, y_pred_fold)
  TP=CML[0,0]
  FP=CML[1,0]
  TN=CML[1,1]
  FN=CML[0,1]

  precision, recall,_=precision_recall_curve(y_test,results.decision_function(Datos_Test.T))

  #Calculamos todas nuestras mediciones
  F1_SCORE.append(2*TP/(2*TP+FP+FN))
  MATTHEWS_C.append((TP*TN-FP*FN)/np.sqrt((TP+FP)*(FN+TN)*(FP+TN)*(FN+TP)))
  ACCURACY.append((TP+TN)/(TP+FN+FP+TN))
  TP_RATE.append(TP/(TP+FN))
  TN_RATE.append(TN/(TN+FP))
  ROC_AUC.append(roc_auc_score(y_test, results.decision_function(Datos_Test.T)))
  PR_AUC.append(auc(recall,precision))

  y_pred.extend(y_pred_fold)
  y_true.extend(y_test)


# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_true,y_pred)

# Convertir la matriz de confusión a un DataFrame para mejor formato
conf_matrix_df = pd.DataFrame(
conf_matrix,
index=['Supervivencia', 'Muerte'],
columns=['Pred. Supervivencia', 'Pred. Muerte']
)

# Agregar totales por fila y por columna
conf_matrix_df.loc['Total fila'] = conf_matrix_df.sum(axis=0)
conf_matrix_df['Total columna'] = conf_matrix_df.sum(axis=1)

# Imprimir la matriz de confusión
print("Matriz de confusión Random CV para SVM:" )
print(conf_matrix_df)

# Calcular la proporción de clasificación errónea
accuracy = accuracy_score(y_true,y_pred)
misclass_rate = 1 - accuracy
print("Prop. error clas. global: %.3f" %  misclass_rate)
print("Media de 100 simulaciones de las mediciones")
print(f"F1 score: {np.mean(F1_SCORE)}")
print(f"Correlación de Matthews: {np.mean(MATTHEWS_C)}")
print(f"Razón de verdaderos positivos (TP-rate): {np.mean(TP_RATE)}")
print(f"Razón de verdaderos negativos (TN-rate): {np.mean(TN_RATE)}")
print(f"Precisión: {np.mean(ACCURACY)}")
print(f"Área bajo la curva ROC (ROC AUC): {np.mean(ROC_AUC)} ")
print(f"Área bajo la curva precision-recall (PR AUC): {np.mean(PR_AUC)}")

ROC_AUC_CV.append(np.mean(ROC_AUC))
TP_RATE_CV.append(np.mean(TP_RATE))
TN_RATE_CV.append(np.mean(TN_RATE))
PR_AUC_CV.append(np.mean(PR_AUC))
F1_SCORE_CV.append(np.mean(F1_SCORE))
ACCURACY_CV.append(np.mean(ACCURACY))
MATTHEWS_C_CV.append(np.mean(MATTHEWS_C))

Matriz de confusión Random CV para SVM:
               Pred. Supervivencia  Pred. Muerte  Total columna
Supervivencia                 3610           486           4096
Muerte                         626          1278           1904
Total fila                    4236          1764           6000
Prop. error clas. global: 0.185
Media de 100 simulaciones de las mediciones
F1 score: 0.8659390610709359
Correlación de Matthews: 0.5617526441183236
Razón de verdaderos positivos (TP-rate): 0.8819575546936455
Razón de verdaderos negativos (TN-rate): 0.6676252503628497
Precisión: 0.8146666666666665
Área bajo la curva ROC (ROC AUC): 0.8579068275134345 
Área bajo la curva precision-recall (PR AUC): 0.7444118252067237


## Clasificación por Regresión Logística

In [ ]:
#Ajustamos el modelo de regresión logística
model=LogisticRegression(max_iter=10000)

#La regresión logística recibe etiquetas como una lista y datos en la forma (No. Datos, No. Caracteristicas)
Datos=X.T
Etiquetas=Y.T
Etiquetas=Etiquetas.ravel()

### K-Fold Cross Validation

In [ ]:
# Listas para almacenar los valores verdaderos y predichos
y_true = []
y_pred = []

# Listas para guardar las mediciones
F1_SCORE=[]
MATTHEWS_C=[]
TP_RATE=[]
TN_RATE=[]
ROC_AUC=[]
PR_AUC=[]
ACCURACY=[]

# Realizar la validación cruzada K-Fold
for train_index, test_index in kf.split(X[0,:]):
  X_train, X_test = Datos[train_index,:], Datos[test_index,:]
  y_train, y_test = Etiquetas[train_index], Etiquetas[test_index]

  # Se estandarizan los datos respecto al conjunto de datos de entrenamiento.
  Datos_estandarizados=Estandarizar(X_train)
  Datos_Test_estandarizados=(X_test-Datos_estandarizados['media'])/Datos_estandarizados['desviacion']

  # Datos reemplazados en las variables binarias
  Datos_Train=reemplazo(Datos_estandarizados['Xstd'].T,train_index)
  Datos_Test=reemplazo(Datos_Test_estandarizados.T,test_index)
  #Entrenamiento del SVM lineal
  results = model.fit(Datos_Train.T, y_train)

  # Predicción de los datos de prueba
  y_pred_fold = results.predict(Datos_Test.T)
  y_score=results.predict_proba(Datos_Test.T)[:,1]

  CML= confusion_matrix(y_test, y_pred_fold)
  TP=CML[0,0]
  FP=CML[1,0]
  TN=CML[1,1]
  FN=CML[0,1]

  precision, recall,_=precision_recall_curve(y_test, y_score)

  #Calculamos todas nuestras mediciones
  F1_SCORE.append(2*TP/(2*TP+FP+FN))
  MATTHEWS_C.append((TP*TN-FP*FN)/np.sqrt((TP+FP)*(FN+TN)*(FP+TN)*(FN+TP)))
  ACCURACY.append((TP+TN)/(TP+FN+FP+TN))
  TP_RATE.append(TP/(TP+FN))
  TN_RATE.append(TN/(TN+FP))
  ROC_AUC.append(roc_auc_score(y_test, y_score))
  PR_AUC.append(auc(recall,precision))

  y_pred.extend(y_pred_fold)
  y_true.extend(y_test)


# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_true,y_pred)

# Convertir la matriz de confusión a un DataFrame para mejor formato
conf_matrix_df = pd.DataFrame(
conf_matrix,
index=['Supervivencia', 'Muerte'],
columns=['Pred. Supervivencia', 'Pred. Muerte']
)

# Agregar totales por fila y por columna
conf_matrix_df.loc['Total fila'] = conf_matrix_df.sum(axis=0)
conf_matrix_df['Total columna'] = conf_matrix_df.sum(axis=1)

# Imprimir la matriz de confusión
print("Matriz de confusión K-Fold CV para la Regresión logística:" )
print(conf_matrix_df)

# Calcular la proporción de clasificación errónea
accuracy = accuracy_score(y_true,y_pred)
misclass_rate = 1 - accuracy
print("Prop. error clas. global: %.3f" %  misclass_rate)
print("Media de 100 simulaciones de las mediciones")
print(f"F1 score: {np.mean(F1_SCORE)}")
print(f"Correlación de Matthews: {np.mean(MATTHEWS_C)}")
print(f"Razón de verdaderos positivos (TP-rate): {np.mean(TP_RATE)}")
print(f"Razón de verdaderos negativos (TN-rate): {np.mean(TN_RATE)}")
print(f"Precisión: {np.mean(ACCURACY)}")
print(f"Area bajo la curva ROC (ROC AUC): {np.mean(ROC_AUC)}" )
print(f"Area bajo la curva precision-recall (PR AUC): {np.mean(PR_AUC)}" )

ROC_AUC_KF.append(np.mean(ROC_AUC))
TP_RATE_KF.append(np.mean(TP_RATE))
TN_RATE_KF.append(np.mean(TN_RATE))
PR_AUC_KF.append(np.mean(PR_AUC))
F1_SCORE_KF.append(np.mean(F1_SCORE))
ACCURACY_KF.append(np.mean(ACCURACY))
MATTHEWS_C_KF.append(np.mean(MATTHEWS_C))

Matriz de confusión K-Fold CV para la Regresión logística:
               Pred. Supervivencia  Pred. Muerte  Total columna
Supervivencia                 3643           417           4060
Muerte                         636          1284           1920
Total fila                    4279          1701           5980
Prop. error clas. global: 0.176
Media de 100 simulaciones de las mediciones
F1 score: 0.8728184885956101
Correlación de Matthews: 0.5859401279581324
Razón de verdaderos positivos (TP-rate): 0.8977102110014825
Razón de verdaderos negativos (TN-rate): 0.6674786120319907
Precisión: 0.8239519774011298
Area bajo la curva ROC (ROC AUC): 0.867760086496239
Area bajo la curva precision-recall (PR AUC): 0.7658876771789277


### Random Cross Validation

In [ ]:
# Listas para almacenar los valores verdaderos y predichos
y_true = []
y_pred = []

# Listas para guardar las mediciones
F1_SCORE=[]
MATTHEWS_C=[]
TP_RATE=[]
TN_RATE=[]
ROC_AUC=[]
PR_AUC=[]
ACCURACY=[]

# Realizar la validación cruzada K-Fold
for train_index, test_index in rcv.split(X[0,:]):
  X_train, X_test = Datos[train_index,:], Datos[test_index,:]
  y_train, y_test = Etiquetas[train_index], Etiquetas[test_index]

  # Se estandarizan los datos respecto al conjunto de datos de entrenamiento.
  Datos_estandarizados=Estandarizar(X_train)
  Datos_Test_estandarizados=(X_test-Datos_estandarizados['media'])/Datos_estandarizados['desviacion']

  # Datos reemplazados en las variables binarias
  Datos_Train=reemplazo(Datos_estandarizados['Xstd'].T,train_index)
  Datos_Test=reemplazo(Datos_Test_estandarizados.T,test_index)
  #Entrenamiento del SVM lineal
  results = model.fit(Datos_Train.T, y_train)

  # Predicción de los datos de prueba
  y_pred_fold = results.predict(Datos_Test.T)
  y_score=results.predict_proba(Datos_Test.T)[:,1]

  CML= confusion_matrix(y_test, y_pred_fold)
  TP=CML[0,0]
  FP=CML[1,0]
  TN=CML[1,1]
  FN=CML[0,1]

  precision, recall,_=precision_recall_curve(y_test, y_score)

  #Calculamos todas nuestras mediciones
  F1_SCORE.append(2*TP/(2*TP+FP+FN))
  MATTHEWS_C.append((TP*TN-FP*FN)/np.sqrt((TP+FP)*(FN+TN)*(FP+TN)*(FN+TP)))
  ACCURACY.append((TP+TN)/(TP+FN+FP+TN))
  TP_RATE.append(TP/(TP+FN))
  TN_RATE.append(TN/(TN+FP))
  ROC_AUC.append(roc_auc_score(y_test, y_score))
  PR_AUC.append(auc(recall,precision))

  y_pred.extend(y_pred_fold)
  y_true.extend(y_test)


# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_true,y_pred)

# Convertir la matriz de confusión a un DataFrame para mejor formato
conf_matrix_df = pd.DataFrame(
conf_matrix,
index=['Supervivencia', 'Muerte'],
columns=['Pred. Supervivencia', 'Pred. Muerte']
)

# Agregar totales por fila y por columna
conf_matrix_df.loc['Total fila'] = conf_matrix_df.sum(axis=0)
conf_matrix_df['Total columna'] = conf_matrix_df.sum(axis=1)

# Imprimir la matriz de confusión
print("Matriz de confusión K-Fold CV para la Regresión logística:" )
print(conf_matrix_df)

# Calcular la proporción de clasificación errónea
accuracy = accuracy_score(y_true,y_pred)
misclass_rate = 1 - accuracy
print("Prop. error clas. global: %.3f" %  misclass_rate)
print("Media de 100 simulaciones de las mediciones")
print(f"F1 score: {np.mean(F1_SCORE)}")
print(f"Correlación de Matthews: {np.mean(MATTHEWS_C)}")
print(f"Razón de verdaderos positivos (TP-rate): {np.mean(TP_RATE)}")
print(f"Razón de verdaderos negativos (TN-rate): {np.mean(TN_RATE)}")
print(f"Precisión: {np.mean(ACCURACY)}")
print(f"Area bajo la curva ROC (ROC AUC): {np.mean(ROC_AUC)}" )
print(f"Area bajo la curva Precision-Recall (PR AUC): {np.mean(PR_AUC)}" )
print("")

ROC_AUC_CV.append(np.mean(ROC_AUC))
TP_RATE_CV.append(np.mean(TP_RATE))
TN_RATE_CV.append(np.mean(TN_RATE))
PR_AUC_CV.append(np.mean(PR_AUC))
F1_SCORE_CV.append(np.mean(F1_SCORE))
ACCURACY_CV.append(np.mean(ACCURACY))
MATTHEWS_C_CV.append(np.mean(MATTHEWS_C))

Matriz de confusión K-Fold CV para la Regresión logística:
               Pred. Supervivencia  Pred. Muerte  Total columna
Supervivencia                 3629           467           4096
Muerte                         621          1283           1904
Total fila                    4250          1750           6000
Prop. error clas. global: 0.181
Media de 100 simulaciones de las mediciones
F1 score: 0.8691193064593971
Correlación de Matthews: 0.5713011976387741
Razón de verdaderos positivos (TP-rate): 0.8867947662992066
Razón de verdaderos negativos (TN-rate): 0.671593731849207
Precisión: 0.8186666666666664
Area bajo la curva ROC (ROC AUC): 0.8624694874348157
Area bajo la curva Precision-Recall (PR AUC): 0.7552607494798461



## SVM kernel

Realizamos ahora el analisis con SVM no lineal

In [ ]:
#Kernel='poly', 'rbf', 'sigmoid'
Gamma=['auto'] #Parametro para las funciones dadas
Degree=[3,4,5] #Solo necesario en polinomial
Coef=[0.001, 0.01,0.1,1] #Necesaria para polinomial y sigmoide

#El SVM recibe etiquetas como una lista y datos en la forma (No. Datos, No. Caracteristicas)
Etiquetas=Y.T
Etiquetas=Etiquetas.ravel()
Datos=X.T

### K-Fold Cross Validation

#### Kernel Polinomial

In [ ]:
for gamma in Gamma:
  for degree in Degree:
    for coef in Coef:
      #Ajustamos el modelo SVM no lineal con kernel polinomial
      model=svm.SVC(kernel='poly', gamma=gamma, degree=degree, coef0=coef)
      # Listas para almacenar los valores verdaderos y predichos
      y_true = []
      y_pred = []

      # Listas para guardar las mediciones
      F1_SCORE=[]
      MATTHEWS_C=[]
      TP_RATE=[]
      TN_RATE=[]
      ROC_AUC=[]
      PR_AUC=[]
      ACCURACY=[]

      # Realizar la validación cruzada K-Fold
      for train_index, test_index in kf.split(X[0,:]):
        X_train, X_test = Datos[train_index,:], Datos[test_index,:]
        y_train, y_test = Etiquetas[train_index], Etiquetas[test_index]

        # Se estandarizan los datos respecto al conjunto de datos de entrenamiento.
        Datos_estandarizados=Estandarizar(X_train)
        Datos_Test_estandarizados=(X_test-Datos_estandarizados['media'])/Datos_estandarizados['desviacion']

        # Datos reemplazados en las variables binarias
        Datos_Train=reemplazo(Datos_estandarizados['Xstd'].T,train_index)
        Datos_Test=reemplazo(Datos_Test_estandarizados.T,test_index)
        #Entrenamiento del SVM kernel
        results = model.fit(Datos_Train.T, y_train)

        # Predicción de los datos de prueba
        y_pred_fold = results.predict(Datos_Test.T)

        CML= confusion_matrix(y_test, y_pred_fold)
        TP=CML[0,0]
        FP=CML[1,0]
        TN=CML[1,1]
        FN=CML[0,1]

        precision,recall,_=precision_recall_curve(y_test, results.decision_function(Datos_Test.T))

        #Calculamos todas nuestras mediciones
        F1_SCORE.append(2*TP/(2*TP+FP+FN))
        MATTHEWS_C.append((TP*TN-FP*FN)/np.sqrt((TP+FP)*(FN+TN)*(FP+TN)*(FN+TP)))
        ACCURACY.append((TP+TN)/(TP+FN+FP+TN))
        TP_RATE.append(TP/(TP+FN))
        TN_RATE.append(TN/(TN+FP))
        ROC_AUC.append(roc_auc_score(y_test, results.decision_function(Datos_Test.T)))
        PR_AUC.append(auc(recall,precision))

        y_pred.extend(y_pred_fold)
        y_true.extend(y_test)


      # Calcular la matriz de confusión
      conf_matrix = confusion_matrix(y_true,y_pred)

      # Convertir la matriz de confusión a un DataFrame para mejor formato
      conf_matrix_df = pd.DataFrame(
      conf_matrix,
      index=['Supervivencia', 'Muerte'],
      columns=['Pred. Supervivencia', 'Pred. Muerte']
      )

      # Agregar totales por fila y por columna
      conf_matrix_df.loc['Total fila'] = conf_matrix_df.sum(axis=0)
      conf_matrix_df['Total columna'] = conf_matrix_df.sum(axis=1)

      # Imprimir la matriz de confusión
      print("Matriz de confusión K-Fold CV para SVM con kernel polinomial" )
      print(f"Gamma: {gamma}, Grado: {degree}, Coef0: {coef}")
      print(conf_matrix_df)

      # Calcular la proporción de clasificación errónea
      accuracy = accuracy_score(y_true,y_pred)
      misclass_rate = 1 - accuracy
      print("Prop. error clas. global: %.3f" %  misclass_rate)
      print("Media de 100 simulaciones de las mediciones")
      print(f"F1 score: {np.mean(F1_SCORE)}")
      print(f"Correlación de Matthews: {np.mean(MATTHEWS_C)}")
      print(f"Razón de verdaderos positivos (TP-rate): {np.mean(TP_RATE)}")
      print(f"Razón de verdaderos negativos (TN-rate): {np.mean(TN_RATE)}")
      print(f"Precisión: {np.mean(ACCURACY)}")
      print(f"Área bajo la curva ROC (ROC AUC): {np.mean(ROC_AUC)}")
      print(f"Área bajo la curva Precision-Recall (PR AUC): {np.mean(PR_AUC)}")
      print("")

      if degree==3 and coef==1:
        ROC_AUC_KF.append(np.mean(ROC_AUC))
        TP_RATE_KF.append(np.mean(TP_RATE))
        TN_RATE_KF.append(np.mean(TN_RATE))
        PR_AUC_KF.append(np.mean(PR_AUC))
        F1_SCORE_KF.append(np.mean(F1_SCORE))
        ACCURACY_KF.append(np.mean(ACCURACY))
        MATTHEWS_C_KF.append(np.mean(MATTHEWS_C))

Matriz de confusión K-Fold CV para SVM con kernel polinomial
Gamma: auto, Grado: 3, Coef0: 0.001
               Pred. Supervivencia  Pred. Muerte  Total columna
Supervivencia                 3862           198           4060
Muerte                        1324           596           1920
Total fila                    5186           794           5980
Prop. error clas. global: 0.255
Media de 100 simulaciones de las mediciones
F1 score: 0.8344795367555787
Correlación de Matthews: 0.36002278499612506
Razón de verdaderos positivos (TP-rate): 0.9518031344844929
Razón de verdaderos negativos (TN-rate): 0.3114889267841684
Precisión: 0.7454774011299434
Área bajo la curva ROC (ROC AUC): 0.8136750265817366
Área bajo la curva Precision-Recall (PR AUC): 0.6889146655230616

Matriz de confusión K-Fold CV para SVM con kernel polinomial
Gamma: auto, Grado: 3, Coef0: 0.01
               Pred. Supervivencia  Pred. Muerte  Total columna
Supervivencia                 3855           205           4060
Muer

#### Kernel Gaussiano

In [ ]:
for gamma in Gamma:
  #Ajustamos el modelo SVM no lineal con kernel gaussiano
  model=svm.SVC(kernel='rbf', gamma=gamma)

  # Listas para almacenar los valores verdaderos y predichos
  y_true = []
  y_pred = []

  # Listas para guardar las mediciones
  F1_SCORE=[]
  MATTHEWS_C=[]
  TP_RATE=[]
  TN_RATE=[]
  ROC_AUC=[]
  PR_AUC=[]
  ACCURACY=[]

  # Realizar la validación cruzada K-Fold
  for train_index, test_index in kf.split(X[0,:]):
    X_train, X_test = Datos[train_index,:], Datos[test_index,:]
    y_train, y_test = Etiquetas[train_index], Etiquetas[test_index]

    # Se estandarizan los datos respecto al conjunto de datos de entrenamiento.
    Datos_estandarizados=Estandarizar(X_train)
    Datos_Test_estandarizados=(X_test-Datos_estandarizados['media'])/Datos_estandarizados['desviacion']

    # Datos reemplazados en las variables binarias
    Datos_Train=reemplazo(Datos_estandarizados['Xstd'].T,train_index)
    Datos_Test=reemplazo(Datos_Test_estandarizados.T,test_index)
    #Entrenamiento del SVM no lineal
    results = model.fit(Datos_Train.T, y_train)

    # Predicción de los datos de prueba
    y_pred_fold = results.predict(Datos_Test.T)

    CML= confusion_matrix(y_test, y_pred_fold)
    TP=CML[0,0]
    FP=CML[1,0]
    TN=CML[1,1]
    FN=CML[0,1]

    precision,recall,_=precision_recall_curve(y_test,results.decision_function(Datos_Test.T))

    #Calculamos todas nuestras mediciones
    F1_SCORE.append(2*TP/(2*TP+FP+FN))
    MATTHEWS_C.append((TP*TN-FP*FN)/np.sqrt((TP+FP)*(FN+TN)*(FP+TN)*(FN+TP)))
    ACCURACY.append((TP+TN)/(TP+FN+FP+TN))
    TP_RATE.append(TP/(TP+FN))
    TN_RATE.append(TN/(TN+FP))
    ROC_AUC.append(roc_auc_score(y_test, results.decision_function(Datos_Test.T)))
    PR_AUC.append(auc(recall,precision))

    y_pred.extend(y_pred_fold)
    y_true.extend(y_test)

  # Calcular la matriz de confusión
  conf_matrix = confusion_matrix(y_true,y_pred)

  # Convertir la matriz de confusión a un DataFrame para mejor formato
  conf_matrix_df = pd.DataFrame(
  conf_matrix,
  index=['Supervivencia', 'Muerte'],
  columns=['Pred. Supervivencia', 'Pred. Muerte']
  )

  # Agregar totales por fila y por columna
  conf_matrix_df.loc['Total fila'] = conf_matrix_df.sum(axis=0)
  conf_matrix_df['Total columna'] = conf_matrix_df.sum(axis=1)

  # Imprimir la matriz de confusión
  print("Matriz de confusión K-Fold CV para SVM con kernel gaussiano" )
  print(f"Gamma: {gamma}")
  print(conf_matrix_df)

  # Calcular la proporción de clasificación errónea
  accuracy = accuracy_score(y_true,y_pred)
  misclass_rate = 1 - accuracy
  print("Prop. error clas. global: %.3f" %  misclass_rate)
  print("Media de 100 simulaciones de las mediciones")
  print(f"F1 score: {np.mean(F1_SCORE)}")
  print(f"Correlación de Matthews: {np.mean(MATTHEWS_C)}")
  print(f"Razón de verdaderos positivos (TP-rate): {np.mean(TP_RATE)}")
  print(f"Razón de verdaderos negativos (TN-rate): {np.mean(TN_RATE)}")
  print(f"Precisión: {np.mean(ACCURACY)}")
  print(f"Área bajo la curva ROC (ROC AUC): {np.mean(ROC_AUC)}")
  print(f"Área bajo la curva Precision-Recall (PR AUC): {np.mean(PR_AUC)}")
  print("")

  ROC_AUC_KF.append(np.mean(ROC_AUC))
  TP_RATE_KF.append(np.mean(TP_RATE))
  TN_RATE_KF.append(np.mean(TN_RATE))
  PR_AUC_KF.append(np.mean(PR_AUC))
  F1_SCORE_KF.append(np.mean(F1_SCORE))
  ACCURACY_KF.append(np.mean(ACCURACY))
  MATTHEWS_C_KF.append(np.mean(MATTHEWS_C))

Matriz de confusión K-Fold CV para SVM con kernel gaussiano
Gamma: auto
               Pred. Supervivencia  Pred. Muerte  Total columna
Supervivencia                 3618           442           4060
Muerte                         764          1156           1920
Total fila                    4382          1598           5980
Prop. error clas. global: 0.202
Media de 100 simulaciones de las mediciones
F1 score: 0.8561077863100454
Correlación de Matthews: 0.5229821607931553
Razón de verdaderos positivos (TP-rate): 0.8918318423437593
Razón de verdaderos negativos (TN-rate): 0.6032865505771953
Precisión: 0.7983587570621468
Área bajo la curva ROC (ROC AUC): 0.853086793806537
Área bajo la curva Precision-Recall (PR AUC): 0.7283806564142459



#### Kernel Sigmoide

In [ ]:
for gamma in Gamma:
  for coef in Coef:
    #Ajustamos el modelo SVM no lineal con kernel sigmoide
    model=svm.SVC(kernel='sigmoid', gamma=gamma, coef0=coef)

    # Listas para almacenar los valores verdaderos y predichos
    y_true = []
    y_pred = []

    # Listas para guardar las mediciones
    F1_SCORE=[]
    MATTHEWS_C=[]
    TP_RATE=[]
    TN_RATE=[]
    ROC_AUC=[]
    PR_AUC=[]
    ACCURACY=[]

    # Realizar la validación cruzada K-Fold
    for train_index, test_index in kf.split(X[0,:]):
      X_train, X_test = Datos[train_index,:], Datos[test_index,:]
      y_train, y_test = Etiquetas[train_index], Etiquetas[test_index]

      # Se estandarizan los datos respecto al conjunto de datos de entrenamiento.
      Datos_estandarizados=Estandarizar(X_train)
      Datos_Test_estandarizados=(X_test-Datos_estandarizados['media'])/Datos_estandarizados['desviacion']

      # Datos reemplazados en las variables binarias
      Datos_Train=reemplazo(Datos_estandarizados['Xstd'].T,train_index)
      Datos_Test=reemplazo(Datos_Test_estandarizados.T,test_index)

      #Entrenamiento del SVM no lineal
      results = model.fit(Datos_Train.T, y_train)

      # Predicción de los datos de prueba
      y_pred_fold = results.predict(Datos_Test.T)

      CML= confusion_matrix(y_test, y_pred_fold)
      TP=CML[0,0]
      FP=CML[1,0]
      TN=CML[1,1]
      FN=CML[0,1]

      precision,recall,_=precision_recall_curve(y_test, results.decision_function(Datos_Test.T))

      #Calculamos todas nuestras mediciones
      F1_SCORE.append(2*TP/(2*TP+FP+FN))
      MATTHEWS_C.append((TP*TN-FP*FN)/np.sqrt((TP+FP)*(FN+TN)*(FP+TN)*(FN+TP)))
      ACCURACY.append((TP+TN)/(TP+FN+FP+TN))
      TP_RATE.append(TP/(TP+FN))
      TN_RATE.append(TN/(TN+FP))
      ROC_AUC.append(roc_auc_score(y_test, results.decision_function(Datos_Test.T)))
      PR_AUC.append(auc(recall,precision))

      y_pred.extend(y_pred_fold)
      y_true.extend(y_test)

    # Calcular la matriz de confusión
    conf_matrix = confusion_matrix(y_true,y_pred)

    # Convertir la matriz de confusión a un DataFrame para mejor formato
    conf_matrix_df = pd.DataFrame(
    conf_matrix,
    index=['Supervivencia', 'Muerte'],
    columns=['Pred. Supervivencia', 'Pred. Muerte']
    )

    # Agregar totales por fila y por columna
    conf_matrix_df.loc['Total fila'] = conf_matrix_df.sum(axis=0)
    conf_matrix_df['Total columna'] = conf_matrix_df.sum(axis=1)

    # Imprimir la matriz de confusión
    print("Matriz de confusión K-Fold CV para SVM con kernel sigmoide" )
    print(f"Gamma: {gamma}, Coef0={coef}")
    print(conf_matrix_df)

    # Calcular la proporción de clasificación errónea
    accuracy = accuracy_score(y_true,y_pred)
    misclass_rate = 1 - accuracy
    print("Prop. error clas. global: %.3f" %  misclass_rate)
    print("Media de 100 simulaciones de las mediciones")
    print(f"F1 score: {np.mean(F1_SCORE)}")
    print(f"Correlación de Matthews: {np.mean(MATTHEWS_C)}")
    print(f"Razón de verdaderos positivos (TP-rate): {np.mean(TP_RATE)}")
    print(f"Razón de verdaderos negativos (TN-rate): {np.mean(TN_RATE)}")
    print(f"Precisión: {np.mean(ACCURACY)}")
    print(f"Área bajo la curva ROC (ROC AUC): {np.mean(ROC_AUC)}")
    print(f"Área bajo la curva Precision-Recall: {np.mean(PR_AUC)}")
    print("")

    if coef==0.01:
      ROC_AUC_KF.append(np.mean(ROC_AUC))
      TP_RATE_KF.append(np.mean(TP_RATE))
      TN_RATE_KF.append(np.mean(TN_RATE))
      PR_AUC_KF.append(np.mean(PR_AUC))
      F1_SCORE_KF.append(np.mean(F1_SCORE))
      ACCURACY_KF.append(np.mean(ACCURACY))
      MATTHEWS_C_KF.append(np.mean(MATTHEWS_C))

Matriz de confusión K-Fold CV para SVM con kernel sigmoide
Gamma: auto, Coef0=0.001
               Pred. Supervivencia  Pred. Muerte  Total columna
Supervivencia                 3628           432           4060
Muerte                         656          1264           1920
Total fila                    4284          1696           5980
Prop. error clas. global: 0.182
Media de 100 simulaciones de las mediciones
F1 score: 0.8686609045157302
Correlación de Matthews: 0.5733148449991028
Razón de verdaderos positivos (TP-rate): 0.8941046450776499
Razón de verdaderos negativos (TN-rate): 0.6594780316006961
Precisión: 0.818104519774011
Área bajo la curva ROC (ROC AUC): 0.8694014452050459
Área bajo la curva Precision-Recall: 0.7639480711113805

Matriz de confusión K-Fold CV para SVM con kernel sigmoide
Gamma: auto, Coef0=0.01
               Pred. Supervivencia  Pred. Muerte  Total columna
Supervivencia                 3634           426           4060
Muerte                         660       

### Random Cross Validation

#### Kernel Polinomial

In [ ]:
for gamma in Gamma:
  for degree in Degree:
    for coef in Coef:
      #Ajustamos el modelo SVM no lineal con kernel polinomial
      model=svm.SVC(kernel='poly', gamma=gamma, degree=degree, coef0=coef)
      # Listas para almacenar los valores verdaderos y predichos
      y_true = []
      y_pred = []

      # Listas para guardar las mediciones
      F1_SCORE=[]
      MATTHEWS_C=[]
      TP_RATE=[]
      TN_RATE=[]
      ROC_AUC=[]
      PR_AUC=[]
      ACCURACY=[]

      # Realizar la validación cruzada K-Fold
      for train_index, test_index in rcv.split(X[0,:]):
        X_train, X_test = Datos[train_index,:], Datos[test_index,:]
        y_train, y_test = Etiquetas[train_index], Etiquetas[test_index]

        # Se estandarizan los datos respecto al conjunto de datos de entrenamiento.
        Datos_estandarizados=Estandarizar(X_train)
        Datos_Test_estandarizados=(X_test-Datos_estandarizados['media'])/Datos_estandarizados['desviacion']

        # Datos reemplazados en las variables binarias
        Datos_Train=reemplazo(Datos_estandarizados['Xstd'].T,train_index)
        Datos_Test=reemplazo(Datos_Test_estandarizados.T,test_index)
        #Entrenamiento del SVM kernel
        results = model.fit(Datos_Train.T, y_train)

        # Predicción de los datos de prueba
        y_pred_fold = results.predict(Datos_Test.T)

        CML= confusion_matrix(y_test, y_pred_fold)
        TP=CML[0,0]
        FP=CML[1,0]
        TN=CML[1,1]
        FN=CML[0,1]

        precision,recall,_=precision_recall_curve(y_test, results.decision_function(Datos_Test.T))

        #Calculamos todas nuestras mediciones
        F1_SCORE.append(2*TP/(2*TP+FP+FN))
        MATTHEWS_C.append((TP*TN-FP*FN)/np.sqrt((TP+FP)*(FN+TN)*(FP+TN)*(FN+TP)))
        ACCURACY.append((TP+TN)/(TP+FN+FP+TN))
        TP_RATE.append(TP/(TP+FN))
        TN_RATE.append(TN/(TN+FP))
        ROC_AUC.append(roc_auc_score(y_test, results.decision_function(Datos_Test.T)))
        PR_AUC.append(auc(recall,precision))

        y_pred.extend(y_pred_fold)
        y_true.extend(y_test)


      # Calcular la matriz de confusión
      conf_matrix = confusion_matrix(y_true,y_pred)

      # Convertir la matriz de confusión a un DataFrame para mejor formato
      conf_matrix_df = pd.DataFrame(
      conf_matrix,
      index=['Supervivencia', 'Muerte'],
      columns=['Pred. Supervivencia', 'Pred. Muerte']
      )

      # Agregar totales por fila y por columna
      conf_matrix_df.loc['Total fila'] = conf_matrix_df.sum(axis=0)
      conf_matrix_df['Total columna'] = conf_matrix_df.sum(axis=1)

      # Imprimir la matriz de confusión
      print("Matriz de confusión Random CV para SVM con kernel polinomial" )
      print(f"Gamma: {gamma}, Grado: {degree}, Coef0: {coef}")
      print(conf_matrix_df)

      # Calcular la proporción de clasificación errónea
      accuracy = accuracy_score(y_true,y_pred)
      misclass_rate = 1 - accuracy
      print("Prop. error clas. global: %.3f" %  misclass_rate)
      print("Media de 100 simulaciones de las mediciones")
      print(f"F1 score: {np.mean(F1_SCORE)}")
      print(f"Correlación de Matthews: {np.mean(MATTHEWS_C)}")
      print(f"Razón de verdaderos positivos (TP-rate): {np.mean(TP_RATE)}")
      print(f"Razón de verdaderos negativos (TN-rate): {np.mean(TN_RATE)}")
      print(f"Precisión: {np.mean(ACCURACY)}")
      print(f"Área bajo la curva ROC (ROC AUC): {np.mean(ROC_AUC)}")
      print(f"Área bajo la curva Precision-Recall (PR AUC): {np.mean(PR_AUC)}")
      print("")

      if degree==3 and coef==1:
        ROC_AUC_CV.append(np.mean(ROC_AUC))
        TP_RATE_CV.append(np.mean(TP_RATE))
        TN_RATE_CV.append(np.mean(TN_RATE))
        PR_AUC_CV.append(np.mean(PR_AUC))
        F1_SCORE_CV.append(np.mean(F1_SCORE))
        ACCURACY_CV.append(np.mean(ACCURACY))
        MATTHEWS_C_CV.append(np.mean(MATTHEWS_C))

Matriz de confusión Random CV para SVM con kernel polinomial
Gamma: auto, Grado: 3, Coef0: 0.001
               Pred. Supervivencia  Pred. Muerte  Total columna
Supervivencia                 3883           213           4096
Muerte                        1303           601           1904
Total fila                    5186           814           6000
Prop. error clas. global: 0.253
Media de 100 simulaciones de las mediciones
F1 score: 0.8356812411651398
Correlación de Matthews: 0.35965405238477305
Razón de verdaderos positivos (TP-rate): 0.9484528611790415
Razón de verdaderos negativos (TN-rate): 0.3177415612640003
Precisión: 0.7473333333333332
Área bajo la curva ROC (ROC AUC): 0.8122754948577894
Área bajo la curva Precision-Recall (PR AUC): 0.6812768978930994

Matriz de confusión Random CV para SVM con kernel polinomial
Gamma: auto, Grado: 3, Coef0: 0.01
               Pred. Supervivencia  Pred. Muerte  Total columna
Supervivencia                 3878           218           4096
Muer

#### Kernel Gaussiano

In [ ]:
for gamma in Gamma:
  #Ajustamos el modelo SVM no lineal con kernel gaussiano
  model=svm.SVC(kernel='rbf', gamma=gamma)

  # Listas para almacenar los valores verdaderos y predichos
  y_true = []
  y_pred = []

  # Listas para guardar las mediciones
  F1_SCORE=[]
  MATTHEWS_C=[]
  TP_RATE=[]
  TN_RATE=[]
  ROC_AUC=[]
  PR_AUC=[]
  ACCURACY=[]

  # Realizar la validación cruzada K-Fold
  for train_index, test_index in rcv.split(X[0,:]):
    X_train, X_test = Datos[train_index,:], Datos[test_index,:]
    y_train, y_test = Etiquetas[train_index], Etiquetas[test_index]

    # Se estandarizan los datos respecto al conjunto de datos de entrenamiento.
    Datos_estandarizados=Estandarizar(X_train)
    Datos_Test_estandarizados=(X_test-Datos_estandarizados['media'])/Datos_estandarizados['desviacion']

    # Datos reemplazados en las variables binarias
    Datos_Train=reemplazo(Datos_estandarizados['Xstd'].T,train_index)
    Datos_Test=reemplazo(Datos_Test_estandarizados.T,test_index)
    #Entrenamiento del SVM no lineal
    results = model.fit(Datos_Train.T, y_train)

    # Predicción de los datos de prueba
    y_pred_fold = results.predict(Datos_Test.T)

    CML= confusion_matrix(y_test, y_pred_fold)
    TP=CML[0,0]
    FP=CML[1,0]
    TN=CML[1,1]
    FN=CML[0,1]

    precision,recall,_=precision_recall_curve(y_test,results.decision_function(Datos_Test.T))

    #Calculamos todas nuestras mediciones
    F1_SCORE.append(2*TP/(2*TP+FP+FN))
    MATTHEWS_C.append((TP*TN-FP*FN)/np.sqrt((TP+FP)*(FN+TN)*(FP+TN)*(FN+TP)))
    ACCURACY.append((TP+TN)/(TP+FN+FP+TN))
    TP_RATE.append(TP/(TP+FN))
    TN_RATE.append(TN/(TN+FP))
    ROC_AUC.append(roc_auc_score(y_test, results.decision_function(Datos_Test.T)))
    PR_AUC.append(auc(recall,precision))

    y_pred.extend(y_pred_fold)
    y_true.extend(y_test)

  # Calcular la matriz de confusión
  conf_matrix = confusion_matrix(y_true,y_pred)

  # Convertir la matriz de confusión a un DataFrame para mejor formato
  conf_matrix_df = pd.DataFrame(
  conf_matrix,
  index=['Supervivencia', 'Muerte'],
  columns=['Pred. Supervivencia', 'Pred. Muerte']
  )

  # Agregar totales por fila y por columna
  conf_matrix_df.loc['Total fila'] = conf_matrix_df.sum(axis=0)
  conf_matrix_df['Total columna'] = conf_matrix_df.sum(axis=1)

  # Imprimir la matriz de confusión
  print("Matriz de confusión K-Fold CV para SVM con kernel gaussiano" )
  print(f"Gamma: {gamma}")
  print(conf_matrix_df)

  # Calcular la proporción de clasificación errónea
  accuracy = accuracy_score(y_true,y_pred)
  misclass_rate = 1 - accuracy
  print("Prop. error clas. global: %.3f" %  misclass_rate)
  print("Media de 100 simulaciones de las mediciones")
  print(f"F1 score: {np.mean(F1_SCORE)}")
  print(f"Correlación de Matthews: {np.mean(MATTHEWS_C)}")
  print(f"Razón de verdaderos positivos (TP-rate): {np.mean(TP_RATE)}")
  print(f"Razón de verdaderos negativos (TN-rate): {np.mean(TN_RATE)}")
  print(f"Precisión: {np.mean(ACCURACY)}")
  print(f"Área bajo la curva ROC (ROC AUC): {np.mean(ROC_AUC)}")
  print(f"Área bajo la curva Precision-Recall (PR AUC): {np.mean(PR_AUC)}")
  print("")

  ROC_AUC_CV.append(np.mean(ROC_AUC))
  TP_RATE_CV.append(np.mean(TP_RATE))
  TN_RATE_CV.append(np.mean(TN_RATE))
  PR_AUC_CV.append(np.mean(PR_AUC))
  F1_SCORE_CV.append(np.mean(F1_SCORE))
  ACCURACY_CV.append(np.mean(ACCURACY))
  MATTHEWS_C_CV.append(np.mean(MATTHEWS_C))

Matriz de confusión K-Fold CV para SVM con kernel gaussiano
Gamma: auto
               Pred. Supervivencia  Pred. Muerte  Total columna
Supervivencia                 3635           461           4096
Muerte                         752          1152           1904
Total fila                    4387          1613           6000
Prop. error clas. global: 0.202
Media de 100 simulaciones de las mediciones
F1 score: 0.8563043797875607
Correlación de Matthews: 0.5149893625199732
Razón de verdaderos positivos (TP-rate): 0.8881628043170395
Razón de verdaderos negativos (TN-rate): 0.6030219781261947
Precisión: 0.7978333333333332
Área bajo la curva ROC (ROC AUC): 0.844482143208372
Área bajo la curva Precision-Recall (PR AUC): 0.7081572088737695



#### Kernel Sigmoide

In [ ]:
for gamma in Gamma:
  for coef in Coef:
    #Ajustamos el modelo SVM no lineal con kernel sigmoide
    model=svm.SVC(kernel='sigmoid', gamma=gamma, coef0=coef)

    # Listas para almacenar los valores verdaderos y predichos
    y_true = []
    y_pred = []

    # Listas para guardar las mediciones
    F1_SCORE=[]
    MATTHEWS_C=[]
    TP_RATE=[]
    TN_RATE=[]
    ROC_AUC=[]
    PR_AUC=[]
    ACCURACY=[]

    # Realizar la validación cruzada K-Fold
    for train_index, test_index in rcv.split(X[0,:]):
      X_train, X_test = Datos[train_index,:], Datos[test_index,:]
      y_train, y_test = Etiquetas[train_index], Etiquetas[test_index]

      # Se estandarizan los datos respecto al conjunto de datos de entrenamiento.
      Datos_estandarizados=Estandarizar(X_train)
      Datos_Test_estandarizados=(X_test-Datos_estandarizados['media'])/Datos_estandarizados['desviacion']

      # Datos reemplazados en las variables binarias
      Datos_Train=reemplazo(Datos_estandarizados['Xstd'].T,train_index)
      Datos_Test=reemplazo(Datos_Test_estandarizados.T,test_index)

      #Entrenamiento del SVM no lineal
      results = model.fit(Datos_Train.T, y_train)

      # Predicción de los datos de prueba
      y_pred_fold = results.predict(Datos_Test.T)

      CML= confusion_matrix(y_test, y_pred_fold)
      TP=CML[0,0]
      FP=CML[1,0]
      TN=CML[1,1]
      FN=CML[0,1]

      precision,recall,_=precision_recall_curve(y_test, results.decision_function(Datos_Test.T))

      #Calculamos todas nuestras mediciones
      F1_SCORE.append(2*TP/(2*TP+FP+FN))
      MATTHEWS_C.append((TP*TN-FP*FN)/np.sqrt((TP+FP)*(FN+TN)*(FP+TN)*(FN+TP)))
      ACCURACY.append((TP+TN)/(TP+FN+FP+TN))
      TP_RATE.append(TP/(TP+FN))
      TN_RATE.append(TN/(TN+FP))
      ROC_AUC.append(roc_auc_score(y_test, results.decision_function(Datos_Test.T)))
      PR_AUC.append(auc(recall,precision))

      y_pred.extend(y_pred_fold)
      y_true.extend(y_test)

    # Calcular la matriz de confusión
    conf_matrix = confusion_matrix(y_true,y_pred)

    # Convertir la matriz de confusión a un DataFrame para mejor formato
    conf_matrix_df = pd.DataFrame(
    conf_matrix,
    index=['Supervivencia', 'Muerte'],
    columns=['Pred. Supervivencia', 'Pred. Muerte']
    )

    # Agregar totales por fila y por columna
    conf_matrix_df.loc['Total fila'] = conf_matrix_df.sum(axis=0)
    conf_matrix_df['Total columna'] = conf_matrix_df.sum(axis=1)

    # Imprimir la matriz de confusión
    print("Matriz de confusión K-Fold CV para SVM con kernel sigmoide" )
    print(f"Gamma: {gamma}, Coef0={coef}")
    print(conf_matrix_df)

    # Calcular la proporción de clasificación errónea
    accuracy = accuracy_score(y_true,y_pred)
    misclass_rate = 1 - accuracy
    print("Prop. error clas. global: %.3f" %  misclass_rate)
    print("Media de 100 simulaciones de las mediciones")
    print(f"F1 score: {np.mean(F1_SCORE)}")
    print(f"Correlación de Matthews: {np.mean(MATTHEWS_C)}")
    print(f"Razón de verdaderos positivos (TP-rate): {np.mean(TP_RATE)}")
    print(f"Razón de verdaderos negativos (TN-rate): {np.mean(TN_RATE)}")
    print(f"Precisión: {np.mean(ACCURACY)}")
    print(f"Área bajo la curva ROC (ROC AUC): {np.mean(ROC_AUC)}")
    print(f"Área bajo la curva Precision-Recall: {np.mean(PR_AUC)}")
    print("")

    if coef==0.001:
      ROC_AUC_CV.append(np.mean(ROC_AUC))
      TP_RATE_CV.append(np.mean(TP_RATE))
      TN_RATE_CV.append(np.mean(TN_RATE))
      PR_AUC_CV.append(np.mean(PR_AUC))
      F1_SCORE_CV.append(np.mean(F1_SCORE))
      ACCURACY_CV.append(np.mean(ACCURACY))
      MATTHEWS_C_CV.append(np.mean(MATTHEWS_C))

Matriz de confusión K-Fold CV para SVM con kernel sigmoide
Gamma: auto, Coef0=0.001
               Pred. Supervivencia  Pred. Muerte  Total columna
Supervivencia                 3633           463           4096
Muerte                         651          1253           1904
Total fila                    4284          1716           6000
Prop. error clas. global: 0.186
Media de 100 simulaciones de las mediciones
F1 score: 0.8663856561581205
Correlación de Matthews: 0.560158329143476
Razón de verdaderos positivos (TP-rate): 0.8876256198990273
Razón de verdaderos negativos (TN-rate): 0.6558830919831727
Precisión: 0.8143333333333332
Área bajo la curva ROC (ROC AUC): 0.8608179979287107
Área bajo la curva Precision-Recall: 0.7498429987717296

Matriz de confusión K-Fold CV para SVM con kernel sigmoide
Gamma: auto, Coef0=0.01
               Pred. Supervivencia  Pred. Muerte  Total columna
Supervivencia                 3616           480           4096
Muerte                         653       

### Resumen

In [ ]:
Mediciones_KF=np.array([MATTHEWS_C_KF, F1_SCORE_KF, ACCURACY_KF, TP_RATE_KF, TN_RATE_KF, PR_AUC_KF, ROC_AUC_KF])
print(MODELOS)
pd.DataFrame(Mediciones_KF.T, index=MODELOS, columns=['MCC', 'F1-Score', 'Accuracy', 'TP rate', 'TN rate', 'PR AUC', 'ROC AUC'])

['ANN TANH', 'ANN RELU', 'ANN SIGMOIDE', 'ANN 2 CAPAS', 'SVM LINEAL', 'SVM POLINOMIAL DEG 3', 'SVM GAUSSIANO', 'SVM SIGMOIDE', 'REGRESION LOG']


,MCC,F1-Score,Accuracy,TP rate,TN rate,PR AUC,ROC AUC
ANN TANH,0.584672,0.873411,0.823932,0.903206,0.657370,0.766370,0.870873
ANN RELU,0.597079,0.875728,0.828630,0.898191,0.680295,0.765758,0.867179
ANN SIGMOIDE,0.571995,0.869228,0.818760,0.894149,0.658193,0.757176,0.862679
ANN 2 CAPAS,0.456811,0.833037,0.768890,0.858277,0.581484,0.696780,0.836887
SVM LINEAL,0.572313,0.867581,0.817579,0.888502,0.666748,0.759646,0.864620
SVM POLINOMIAL DEG 3,0.585940,0.872818,0.823952,0.897710,0.667479,0.765888,0.867760
SVM GAUSSIANO,0.449270,0.830517,0.765410,0.855192,0.578969,0.653870,0.796592
SVM SIGMOIDE,0.522982,0.856108,0.798359,0.891832,0.603287,0.728381,0.853087
REGRESION LOG,0.573967,0.868978,0.818438,0.895480,0.658260,0.761587,0.869107


In [ ]:
Mediciones_CV=np.array([MATTHEWS_C_CV, F1_SCORE_CV, ACCURACY_CV, TP_RATE_CV, TN_RATE_CV, PR_AUC_CV, ROC_AUC_CV])
pd.DataFrame(Mediciones_CV.T, index=MODELOS, columns=['MCC', 'F1-Score', 'Accuracy', 'TP rate', 'TN rate', 'PR AUC', 'ROC AUC'])

,MCC,F1-Score,Accuracy,TP rate,TN rate,PR AUC,ROC AUC
ANN TANH,0.568751,0.869755,0.818333,0.894750,0.654868,0.754708,0.864856
ANN RELU,0.578446,0.871056,0.821833,0.887568,0.678463,0.747974,0.858551
ANN SIGMOIDE,0.558535,0.865526,0.813500,0.885026,0.659169,0.743584,0.854770
ANN 2 CAPAS,0.443832,0.831575,0.765833,0.854671,0.574132,0.689993,0.834299
SVM LINEAL,0.561753,0.865939,0.814667,0.881958,0.667625,0.744412,0.857907
SVM POLINOMIAL DEG 3,0.571301,0.869119,0.818667,0.886795,0.671594,0.755261,0.862469
SVM GAUSSIANO,0.434054,0.828196,0.761500,0.850640,0.569062,0.640759,0.790590
SVM SIGMOIDE,0.514989,0.856304,0.797833,0.888163,0.603022,0.708157,0.844482
REGRESION LOG,0.560158,0.866386,0.814333,0.887626,0.655883,0.749843,0.860818
